In [12]:
# Librerías.
import pandas as pd
import Funciones as f
import unidecode

In [ ]:
%%capture
%run "14. Ordenamiento de columnas.ipynb"

In [14]:
# Eliminamos los datos de los primeros IP Items asociados y rellenamos con la mediana de la población por Categoría.
for Nombre_df, df in dfs_Finales.items():
    dfs_Finales[Nombre_df] = f.Eliminar_Primeros_Datos_IP_Items_Asociados(df)

In [15]:
# CONTROL ACTUALIZADO: Con Orden_IP_Items_Asociados + verificación IP no asociados.
print("\n🔧 CONTROL ACTUALIZADO: FUNCIÓN MODIFICADA + VERIFICACIÓN IP NO ASOCIADOS")
print("="*80)

# 1. Verificar que se usan 3 números únicos por participante.
print("1. VERIFICACIÓN DE 3 NÚMEROS ÚNICOS POR PARTICIPANTE:")

if 'Orden_IP_Items_Asociados' in df.columns:
   Casos_Con_3_Unicos = 0
   Casos_Con_Menos_De_3 = 0
   Total_Casos_Verificados = 0
   Ejemplos_Proceso = []
   
   # Analizar cada participante.
   for Index, Fila in df.iterrows():
       Orden_IP_Items = Fila['Orden_IP_Items_Asociados']
       ID_Participante = Fila['ID']
       
       if isinstance(Orden_IP_Items, list) and len(Orden_IP_Items) > 0:
           Total_Casos_Verificados += 1
           
           # Simular el proceso de la función modificada.
           Numeros_IP_Unicos = set()
           Items_Procesados = []
           
           for Item in Orden_IP_Items:
               if isinstance(Item, str) and '_' in Item:
                   Numero = Item.split('_')[0]
                   Items_Procesados.append(Item)
                   Numeros_IP_Unicos.add(Numero)
                   
                   if len(Numeros_IP_Unicos) >= 3:
                       break
           
           # Verificar resultado.
           if len(Numeros_IP_Unicos) >= 3:
               Casos_Con_3_Unicos += 1
           else:
               Casos_Con_Menos_De_3 += 1
           
           # Guardar ejemplos para mostrar (incluyendo orden completo).
           if len(Ejemplos_Proceso) < 5:
               Ejemplos_Proceso.append({
                   'ID': ID_Participante,
                   'Orden_Completo': Orden_IP_Items,  # NUEVO: Orden completo
                   'Items_Procesados': Items_Procesados,
                   'Numeros_Unicos': sorted(list(Numeros_IP_Unicos)),
                   'Cantidad_Procesada': len(Items_Procesados),
                   'Logro_3_Unicos': len(Numeros_IP_Unicos) >= 3
               })
   
   # Mostrar resultados.
   if Total_Casos_Verificados > 0:
       Porcentaje_Exitosos = (Casos_Con_3_Unicos / Total_Casos_Verificados) * 100
       
       print(f"   Total casos verificados: {Total_Casos_Verificados}")
       print(f"   Casos con 3+ números únicos: {Casos_Con_3_Unicos} ({Porcentaje_Exitosos:.1f}%)")
       print(f"   Casos con <3 números únicos: {Casos_Con_Menos_De_3}")
       
       # Mostrar ejemplos del proceso CON ORDEN COMPLETO.
       print("\n   Ejemplos detallados del proceso de selección:")
       for i, Ejemplo in enumerate(Ejemplos_Proceso, 1):
           print(f"\n     Caso {i} - ID {Ejemplo['ID']}:")
           print(f"       🔄 Orden_IP_Items_Asociados completo:")
           print(f"          {Ejemplo['Orden_Completo']}")
           print(f"       📋 Ítems procesados hasta obtener 3 únicos:")
           print(f"          {Ejemplo['Items_Procesados']}")
           print(f"       🎯 Números únicos obtenidos: {Ejemplo['Numeros_Unicos']}")
           print(f"       📊 Cantidad de ítems procesados: {Ejemplo['Cantidad_Procesada']}")
           Estado = "✅ Éxito" if Ejemplo['Logro_3_Unicos'] else "❌ Insuficiente"
           print(f"       🏁 Resultado: {Estado}")
   
else:
   print("   ❌ Columna 'Orden_IP_Items_Asociados' no encontrada")

# 2. VERIFICACIÓN CRÍTICA: IP ITEMS NO ASOCIADOS NO DEBEN SER ELIMINADOS.
print(f"\n2. VERIFICACIÓN CRÍTICA: IP ITEMS NO ASOCIADOS (PRESERVADOS):")

# Identificar columnas IP no asociadas (formato: IP_Item_X_Respuesta).
Columnas_IP_No_Asociados = [col for col in df.columns 
                          if 'IP_Item_' in col and '_Respuesta' in col 
                          and '_Izq_' not in col and '_Der_' not in col]

# Identificar columnas IP asociados (formato: IP_Item_X_Izq/Der_Respuesta).
Columnas_IP_Asociados = [col for col in df.columns 
                       if 'IP_Item_' in col and '_Respuesta' in col 
                       and ('_Izq_' in col or '_Der_' in col)]

print(f"   IP Items NO asociados encontrados: {len(Columnas_IP_No_Asociados)}")
print(f"   IP Items asociados encontrados: {len(Columnas_IP_Asociados)}")

if len(Columnas_IP_No_Asociados) > 0:
   print(f"\n   Columnas IP NO asociados (deben estar PRESERVADAS):")
   for col in Columnas_IP_No_Asociados[:8]:  # Mostrar primeras 8.
       print(f"     - {col}")
   if len(Columnas_IP_No_Asociados) > 8:
       print(f"     ... y {len(Columnas_IP_No_Asociados) - 8} más")
   
   # Verificar que NO fueron eliminados.
   print(f"\n   Verificación de preservación de IP NO asociados:")
   print(f"   Columna                          | % Datos Presentes | Estado")
   print(f"   " + "-"*65)
   
   IPs_No_Asociados_Preservados = 0
   
   for col in Columnas_IP_No_Asociados[:10]:  # Verificar primeros 10.
       Datos_Presentes = df[col].notna().sum()
       Porcentaje_Presente = (Datos_Presentes / len(df)) * 100
       
       if Porcentaje_Presente >= 80:  # Al menos 80% de datos presentes.
           Estado = "✅ PRESERVADO"
           IPs_No_Asociados_Preservados += 1
       elif Porcentaje_Presente >= 50:
           Estado = "⚠️  Parcialmente eliminado"
       else:
           Estado = "❌ MAYORÍA ELIMINADA"
       
       print(f"   {col:<32} | {Porcentaje_Presente:15.1f}% | {Estado}")
   
   # Resumen de preservación.
   if len(Columnas_IP_No_Asociados) > 0:
       Porcentaje_Preservacion = (IPs_No_Asociados_Preservados / min(10, len(Columnas_IP_No_Asociados))) * 100
       print(f"\n   📊 Resumen de preservación de IP NO asociados:")
       print(f"     Columnas bien preservadas: {IPs_No_Asociados_Preservados}/{min(10, len(Columnas_IP_No_Asociados))} ({Porcentaje_Preservacion:.1f}%)")
       
       if Porcentaje_Preservacion >= 90:
           print(f"     ✅ EXCELENTE: IP no asociados correctamente preservados")
       elif Porcentaje_Preservacion >= 70:
           print(f"     📊 BUENO: Mayoría de IP no asociados preservados")
       else:
           print(f"     ❌ ERROR: IP no asociados están siendo eliminados incorrectamente")

# 3. RECUENTO CORRECTO: Solo IP Items Asociados para el conteo de 14 datos.
print(f"\n3. RECUENTO CORRECTO DE DATOS FINALES (SOLO IP ASOCIADOS):")
print(f"   Objetivo: 14 datos por participante de IP Items ASOCIADOS únicamente")

# Buscar columnas IP asociados _Respuesta y _Tiempo.
Columnas_Asociados_Respuesta = [col for col in df.columns if 'IP_Item_' in col and '_Respuesta' in col 
                              and ('_Izq_' in col or '_Der_' in col)]
Columnas_Asociados_Tiempo = [col for col in df.columns if 'IP_Item_' in col and '_Tiempo' in col 
                           and ('_Izq_' in col or '_Der_' in col)]

print(f"   Columnas IP asociados _Respuesta: {len(Columnas_Asociados_Respuesta)}")
print(f"   Columnas IP asociados _Tiempo: {len(Columnas_Asociados_Tiempo)}")

if len(Columnas_Asociados_Respuesta) > 0 and len(Columnas_Asociados_Tiempo) > 0:
   # Analizar una muestra de participantes CON DETALLE.
   Muestra_Participantes = df.iloc[:8]  # Primeros 8 participantes.
   
   print(f"\n   Análisis detallado de muestra (primeros 8 participantes):")
   
   for i, (_, Fila) in enumerate(Muestra_Participantes.iterrows(), 1):
       ID_Participante = Fila['ID']
       Orden_IP_Items = Fila.get('Orden_IP_Items_Asociados', 'N/A')
       
       print(f"\n     Participante {i} - ID {ID_Participante}:")
       print(f"       🔄 Orden_IP_Items_Asociados: {Orden_IP_Items}")
       
       # Contar datos IP asociados presentes.
       Datos_Respuesta = sum(1 for col in Columnas_Asociados_Respuesta if pd.notna(Fila[col]))
       Datos_Tiempo = sum(1 for col in Columnas_Asociados_Tiempo if pd.notna(Fila[col]))
       Total_Datos_Asociados = Datos_Respuesta + Datos_Tiempo
       
       # Contar datos IP NO asociados (para verificar que no fueron tocados).
       Datos_No_Asociados = sum(1 for col in Columnas_IP_No_Asociados if pd.notna(Fila[col]))
       
       print(f"       📊 IP Asociados - _Respuesta: {Datos_Respuesta}")
       print(f"       📊 IP Asociados - _Tiempo: {Datos_Tiempo}")
       print(f"       🎯 Total IP Asociados: {Total_Datos_Asociados}")
       print(f"       🔒 IP NO Asociados preservados: {Datos_No_Asociados}")
       
       # Estado del objetivo 14 datos.
       if Total_Datos_Asociados == 14:
           Estado_14 = "✅ PERFECTO (14 datos)"


🔧 CONTROL ACTUALIZADO: FUNCIÓN MODIFICADA + VERIFICACIÓN IP NO ASOCIADOS
1. VERIFICACIÓN DE 3 NÚMEROS ÚNICOS POR PARTICIPANTE:
   Total casos verificados: 1294
   Casos con 3+ números únicos: 1294 (100.0%)
   Casos con <3 números únicos: 0

   Ejemplos detallados del proceso de selección:

     Caso 1 - ID 5749:
       🔄 Orden_IP_Items_Asociados completo:
          ['25_Izq', '3_Izq', '27_Izq', '29_Der', '8_Izq', '5_Der', '28_Izq', '8_Der', '11_Der', '3_Der', '7_Izq', '5_Izq', '28_Der', '24_Der', '29_Izq', '27_Der', '24_Izq', '11_Izq', '25_Der', '7_Der']
       📋 Ítems procesados hasta obtener 3 únicos:
          ['25_Izq', '3_Izq', '27_Izq']
       🎯 Números únicos obtenidos: ['25', '27', '3']
       📊 Cantidad de ítems procesados: 3
       🏁 Resultado: ✅ Éxito

     Caso 2 - ID 5756:
       🔄 Orden_IP_Items_Asociados completo:
          ['28_Izq', '8_Izq', '22_Der', '22_Izq', '11_Der', '8_Der', '9_Der', '16_Izq', '30_Der', '25_Der', '16_Der', '9_Izq', '28_Der', '11_Izq', '6_Izq', '6

In [ ]:
# VERIFICACIÓN CORREGIDA: 14 datos válidos restantes después de eliminación.
for Nombre_df, df in dfs_Finales.items():
   print(f"\n🔍 VERIFICACIÓN CORRECTA: 14 DATOS VÁLIDOS RESTANTES - {Nombre_df}")
   print("="*80)

   # 5. DETECCIÓN Y ANÁLISIS DE CASOS CON NÚMEROS REPETIDOS.
   print("5. DETECCIÓN DE CASOS CON NÚMEROS REPETIDOS EN ORDEN:")

   if 'Orden_IP_Items_Asociados' in df.columns:
       Casos_Con_Repetidos = []
       Casos_Sin_Repetidos = 0
       Total_Casos_Analizados = 0
   
       # Analizar cada participante buscando repetidos.
       for Index, Fila in df.iterrows():
           Orden_IP_Items = Fila['Orden_IP_Items_Asociados']
           ID_Participante = Fila['ID']
           
           if isinstance(Orden_IP_Items, list) and len(Orden_IP_Items) > 0:
               Total_Casos_Analizados += 1
               
               # Extraer números de toda la lista.
               Todos_Los_Numeros = []
               for Item in Orden_IP_Items:
                   if isinstance(Item, str) and '_' in Item:
                       Numero = Item.split('_')[0]
                       Todos_Los_Numeros.append(Numero)
           
               # Verificar si hay repetidos en los primeros elementos.
               Primeros_3_Numeros = Todos_Los_Numeros[:3]
               Numeros_Unicos_3 = set(Primeros_3_Numeros)
               
               if len(Primeros_3_Numeros) != len(Numeros_Unicos_3):
                   # Hay repetidos en los primeros 3.
                   from collections import Counter
                   Contador = Counter(Primeros_3_Numeros)
                   Repetidos = [num for num, count in Contador.items() if count > 1]
                   
                   # Simular proceso de la función hasta obtener 3 únicos.
                   Numeros_Procesados = []
                   Items_Procesados = []
                   Unicos_Obtenidos = set()
               
                   for Item in Orden_IP_Items:
                       if isinstance(Item, str) and '_' in Item:
                           Numero = Item.split('_')[0]
                           Numeros_Procesados.append(Numero)
                           Items_Procesados.append(Item)
                           Unicos_Obtenidos.add(Numero)
                           
                           if len(Unicos_Obtenidos) >= 3:
                               break
               
                   Casos_Con_Repetidos.append({
                       'ID': ID_Participante,
                       'Orden_Completo': Orden_IP_Items,
                       'Todos_Numeros': Todos_Los_Numeros,
                       'Primeros_3_Numeros': Primeros_3_Numeros,
                       'Repetidos_En_3': Repetidos,
                       'Items_Procesados_Hasta_3': Items_Procesados,
                       'Numeros_Procesados_Hasta_3': Numeros_Procesados,
                       'Numeros_A_Eliminar': sorted(list(Unicos_Obtenidos)),
                       'Expansion_Necesaria': len(Items_Procesados) - 3
                   })
               else:
                   Casos_Sin_Repetidos += 1
   
       # Mostrar estadísticas.
       if Total_Casos_Analizados > 0:
           Cantidad_Con_Repetidos = len(Casos_Con_Repetidos)
           Porcentaje_Repetidos = (Cantidad_Con_Repetidos / Total_Casos_Analizados) * 100
           
           print(f"   Total casos analizados: {Total_Casos_Analizados}")
           print(f"   Casos SIN números repetidos (primeros 3): {Casos_Sin_Repetidos}")
           print(f"   Casos CON números repetidos (primeros 3): {Cantidad_Con_Repetidos} ({Porcentaje_Repetidos:.1f}%)")
           
           # VERIFICACIÓN ESPECÍFICA: 14 DATOS VÁLIDOS + ELIMINACIÓN CORRECTA.
           if len(Casos_Con_Repetidos) > 0:
               print(f"\n   📋 VERIFICACIÓN ESPECÍFICA: 14 DATOS VÁLIDOS + ELIMINACIÓN:")
               
               # Identificar todas las columnas IP asociados.
               Columnas_IP_Asociados_Respuesta = [col for col in df.columns if 'IP_Item_' in col and '_Respuesta' in col and ('_Izq_' in col or '_Der_' in col)]
               Columnas_IP_Asociados_Tiempo = [col for col in df.columns if 'IP_Item_' in col and '_Tiempo' in col and ('_Izq_' in col or '_Der_' in col)]
               
               for i, Caso in enumerate(Casos_Con_Repetidos[:3], 1):  # Verificar primeros 3 casos.
                   ID_Participante = Caso['ID']
                   Numeros_A_Eliminar = Caso['Numeros_A_Eliminar']
                   
                   print(f"\n     🔍 CASO {i} - ID {ID_Participante}:")
                   print(f"       🔄 Orden completo: {Caso['Orden_Completo']}")
                   print(f"       🔢 Primeros 3 números: {Caso['Primeros_3_Numeros']}")
                   print(f"       🔁 Números repetidos: {Caso['Repetidos_En_3']}")
                   print(f"       ⚙️  Items procesados: {Caso['Items_Procesados_Hasta_3']}")
                   print(f"       🎯 Números que DEBEN estar eliminados: {Numeros_A_Eliminar}")
                   print(f"       📈 Expansión requerida: +{Caso['Expansion_Necesaria']} ítems")
                   
                   # Buscar la fila del participante.
                   Fila_Participante = df[df['ID'] == ID_Participante].iloc[0]
                   
                   # 1. CONTAR TODOS LOS DATOS VÁLIDOS RESTANTES (OBJETIVO: 14).
                   Datos_Validos_Restantes = 0
                   for col in Columnas_IP_Asociados_Respuesta + Columnas_IP_Asociados_Tiempo:
                       if pd.notna(Fila_Participante[col]):
                           Datos_Validos_Restantes += 1
                   
                   print(f"\n       📊 DATOS VÁLIDOS RESTANTES TOTAL: {Datos_Validos_Restantes}")
                   
                   if Datos_Validos_Restantes == 14:
                       print(f"       ✅ PERFECTO: Exactamente 14 datos válidos restantes")
                   elif Datos_Validos_Restantes > 14:
                       print(f"       ⚠️ EXCESO: {Datos_Validos_Restantes} datos válidos (+{Datos_Validos_Restantes - 14})")
                   else:
                       print(f"       ❌ DÉFICIT: {Datos_Validos_Restantes} datos válidos (-{14 - Datos_Validos_Restantes})")
                   
                   # 2. VERIFICAR QUE LOS NÚMEROS A ELIMINAR ESTÉN EFECTIVAMENTE ELIMINADOS.
                   print(f"\n       🚫 VERIFICACIÓN DE ELIMINACIÓN DE NÚMEROS SELECCIONADOS:")
                   
                   Eliminaciones_Correctas = 0
                   Eliminaciones_Esperadas = 0
                   
                   for Numero in Numeros_A_Eliminar:
                       print(f"         📋 IP_Item_{Numero} (DEBE estar eliminado):")
                       
                       Eliminado_Completo = True
                       Datos_Eliminados_Este_Numero = 0
                       Datos_Totales_Este_Numero = 0
                       
                       for Direccion in ['_Izq', '_Der']:
                           # Verificar _Respuesta.
                           Col_Respuesta = f'IP_Item_{Numero}{Direccion}_Respuesta'
                           if Col_Respuesta in df.columns:
                               Datos_Totales_Este_Numero += 1
                               Eliminaciones_Esperadas += 1
                               Valor_Resp = Fila_Participante[Col_Respuesta]
                               if pd.isna(Valor_Resp):
                                   Datos_Eliminados_Este_Numero += 1
                                   Eliminaciones_Correctas += 1
                                   Estado_Resp = "✅ ELIMINADO"
                               else:
                                   Eliminado_Completo = False
                                   Estado_Resp = f"❌ NO ELIMINADO: {Valor_Resp}"
                               print(f"           {Col_Respuesta}: {Estado_Resp}")
                           
                           # Verificar _Tiempo.
                           Col_Tiempo = f'IP_Item_{Numero}{Direccion}_Tiempo'
                           if Col_Tiempo in df.columns:
                               Datos_Totales_Este_Numero += 1
                               Eliminaciones_Esperadas += 1
                               Valor_Tiempo = Fila_Participante[Col_Tiempo]
                               if pd.isna(Valor_Tiempo):
                                   Datos_Eliminados_Este_Numero += 1
                                   Eliminaciones_Correctas += 1
                                   Estado_Tiempo = "✅ ELIMINADO"
                               else:
                                   Eliminado_Completo = False
                                   Estado_Tiempo = f"❌ NO ELIMINADO: {Valor_Tiempo}"
                               print(f"           {Col_Tiempo}: {Estado_Tiempo}")
                       
                       # Resumen por número.
                       if Datos_Totales_Este_Numero > 0:
                           Porcentaje_Eliminado = (Datos_Eliminados_Este_Numero / Datos_Totales_Este_Numero) * 100
                           if Eliminado_Completo:
                               print(f"         ✅ IP_Item_{Numero}: COMPLETAMENTE ELIMINADO")
                           else:
                               print(f"         ❌ IP_Item_{Numero}: ELIMINACIÓN INCOMPLETA ({Porcentaje_Eliminado:.1f}%)")
                   
                   # 3. RESUMEN FINAL PARA ESTE CASO.
                   print(f"\n       🏁 RESUMEN FINAL CASO {i}:")
                   print(f"         📊 Datos válidos restantes: {Datos_Validos_Restantes}/14")
                   
                   if Eliminaciones_Esperadas > 0:
                       Precision_Eliminacion = (Eliminaciones_Correctas / Eliminaciones_Esperadas) * 100
                       print(f"         🎯 Precisión de eliminación: {Precision_Eliminacion:.1f}% ({Eliminaciones_Correctas}/{Eliminaciones_Esperadas})")
                   
                   # EVALUACIÓN FINAL.
                   if Datos_Validos_Restantes == 14 and Precision_Eliminacion == 100:
                       print(f"         🏆 EXCELENTE: Caso procesado perfectamente")
                   elif Datos_Validos_Restantes == 14:
                       print(f"         📊 BUENO: 14 datos válidos, pero eliminación incompleta")
                   elif Precision_Eliminacion == 100:
                       print(f"         ⚠️ PARCIAL: Eliminación correcta, pero datos válidos ≠ 14")
                   else:
                       print(f"         ❌ REVISAR: Problemas en datos válidos y eliminación")
                   
                   print(f"         " + "-"*50)
           
           else:
               print(f"   ✅ PERFECTO: No se encontraron casos con números repetidos")

   print(f"\n" + "="*60)
   print(f"✅ VERIFICACIÓN CORRECTA COMPLETADA")
   print(f"="*60)

In [17]:
# Exportar base para el control.
for Nombre_df, df in dfs_Finales.items():
    dfs_Finales[Nombre_df].head(50).to_excel(f'Controles/15. Eliminación de primeros ítems ({Nombre_df}).xlsx', index=False)